After the End of World Cup, we know that Argentina of the legendary Lionel Messi was celebreted the title of world champion for the third time. But, if the World Cup had other way to win the title, Argentina will win too?
The idea of this notebook is use data to play with this possibilites. To discover the answers, i will use a database with all national teams matches, since the first friendly match in 1887(!!!)

I will propose diferentes formats:
1.   Knockout round (Loose the match, is out of the competition)
2.   League format (The team plays all the other 31 teams);
3.   League format in two legs (The team plays all the other 31 teams, in home and away)
4.   Swiss model

# Imports

In [ ]:
import pandas as pd
from datetime import date
from datetime import datetime
import itertools

In [ ]:
df_matches = pd.read_csv('/content/drive/MyDrive/dados/results.csv')
df_shootouts = pd.read_csv('/content/drive/MyDrive/dados/shootouts.csv')
older_date = date.today()

In [ ]:
df_matches.tail(10)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
44361,2022-12-30,Oman,Syria,1,0,Friendly,Dubai,United Arab Emirates,True
44362,2022-12-30,United Arab Emirates,Lebanon,1,0,Friendly,Abu Dhabi,United Arab Emirates,False
44363,2023-01-08,Iceland,Estonia,1,1,Friendly,Albufeira,Portugal,True
44364,2023-01-09,Sweden,Finland,2,0,Friendly,Faro-Loulé,Portugal,True
44365,2023-01-12,Finland,Estonia,0,1,Friendly,Albufeira,Portugal,True
44366,2023-01-12,Sweden,Iceland,2,1,Friendly,Faro-Loulé,Portugal,True
44367,2023-01-25,United States,Serbia,1,2,Friendly,Los Angeles,United States,False
44368,2023-01-28,United States,Colombia,0,0,Friendly,Carson,United States,False
44369,2023-02-22,Grenada,Barbados,1,1,Friendly,Sauteurs,Grenada,False
44370,2023-02-26,Grenada,Barbados,2,2,Friendly,St. George's,Grenada,False


In [ ]:
df_shootouts.tail(20)

,date,home_team,away_team,winner
527,2022-01-26,Mali,Equatorial Guinea,Equatorial Guinea
528,2022-02-03,Cameroon,Egypt,Egypt
529,2022-02-05,Cameroon,Burkina Faso,Cameroon
530,2022-02-06,Senegal,Egypt,Senegal
531,2022-03-25,Tajikistan,Uganda,Uganda
532,2022-03-29,Kazakhstan,Moldova,Kazakhstan
533,2022-03-29,Senegal,Egypt,Senegal
534,2022-06-13,Australia,Peru,Australia
535,2022-06-14,Chile,Ghana,Ghana
536,2022-09-22,Thailand,Malaysia,Malaysia


# Main Functions

In [ ]:

def search_matches(df_matches,th,ta):
  df_matches_returned = df_matches.loc[((df_matches["home_team"] == th) & (df_matches["away_team"] == ta)) | ((df_matches["home_team"] == ta) & (df_matches["away_team"] == th))]
  return df_matches_returned

def search_shootouts(df_shootouts,th,ta):
  df_shootouts_returned = df_shootouts.loc[((df_shootouts["home_team"] == th) & (df_shootouts["away_team"] == ta)) | ((df_shootouts["home_team"] == ta) & (df_shootouts["away_team"] == th))]
  return df_shootouts_returned

def breaking_draw(df_matches,th,ta):
  df_matches_returned = search_matches(df_matches.copy(),th,ta)
  df_matches_returned_no_draw = df_matches_returned.loc[df_matches_returned["home_score"]!=df_matches_returned["away_score"]]
  if len(df_matches_returned)>1:
    df_winner = df_matches_returned_no_draw.tail(1)
    home_team = df_winner['home_team'].iloc[0]
    away_team = df_winner['away_team'].iloc[0]
    home_score = df_winner['home_score'].iloc[0]
    away_score = df_winner['away_score'].iloc[0]
    historic_date = df_winner['date'].iloc[0]
    print("Historic last result:")
    print("Result:",home_team,"-",home_score,"X",away_score,"-",away_team)
    print("Date historic:", historic_date)
    if home_score > away_score:
      t_winner = home_team
    else:
      t_winner = away_team
  else:
    print(th," and ",ta," without winner registers")
    t_winner = "No historic"
  return t_winner, historic_date


def define_shootouts_winner(df_shootouts,th,ta):
  print("Shootouts: ",th," X ",ta)
  df_shootouts_returned = search_shootouts(df_shootouts.copy(),th,ta)
  if len(df_shootouts_returned)>0:
    df_winner = df_shootouts_returned.tail(1)
    shootout_date = df_shootouts_returned['date'].iloc[0]
    print("date:",shootout_date)
    t_winner = df_winner['winner'].iloc[0]
  else:
    print(th," and ",ta," never shootouts")
    t_winner = 'draw'
    shootout_date = date.today()
  return t_winner,shootout_date

def define_winner(df_matches, df_shootouts,th,ta,conditions,older_date):
  print("MATCH: ",th," X ",ta)
  df_matches_returned = search_matches(df_matches.copy(),th,ta)
  df_matches_returned_wc = df_matches_returned[df_matches_returned["tournament"]=="FIFA World Cup"]
  if len(df_matches_returned_wc)>0:
    df_winner = df_matches_returned_wc.tail(1)
  elif len(df_matches_returned)>0:
    df_winner = df_matches_returned.tail(1)
  else:
    print(th," and ",ta," never played")
    if "same opponent" in conditions:
      df_winner = in_case_no_match(df_matches,teamA, teamB)
    else:
      t_winner = "No matches"
  #####
  home_team = df_winner['home_team'].iloc[0]
  away_team = df_winner['away_team'].iloc[0]
  home_score = df_winner['home_score'].iloc[0]
  away_score = df_winner['away_score'].iloc[0]
  print("Result:",home_team,"-",home_score,"X",away_score,"-",away_team)
  if home_score > away_score:
    t_winner = home_team
  elif home_score < away_score:
    t_winner = away_team
  else:
    if "no draw" in conditions:
      t_winner,date_register = define_shootouts_winner(df_shootouts.copy(),th,ta)
      if(t_winner == "draw"):
        t_winner, date_register = breaking_draw(df_matches,th,ta)
    #date_register = date(date_register,'%Y-%m-%d')
    print(type(older_date))
    print(type(date_register))
    #if (date_register < older_date):
      #older_date = date_register

  return t_winner, older_date




# Pre-processing

In [ ]:
national_teams = {"A1": "Qatar", "A2": "Ecuador", "A3": "Senegal","A4": "Netherlands",
         "B1": "England", "B2": "Iran", "B3": "USA","B4": "Wales",
         "C1": "Argentina", "C2": "Saudi Arabia", "C3": "Mexico","C4": "Poland",
         "D1": "France", "D2": "Australia", "D3": "Denmark","D4": "Tunisia",
         "E1": "Spain", "E2": "Costa Rica", "E3": "Germany","E4": "Japan",
         "F1": "Belgium", "F2": "Canada", "F3": "Morocco","F4": "Croatia",
         "G1": "Brazil", "G2": "Serbia", "G3": "Switzerland","G4": "Cameroon",
         "H1": "Portugal", "H2": "Ghana", "H3": "Uruguay","H4": "South Korea"}

In [ ]:
df_matches['home_score'] = df_matches['home_score'].astype(int)
df_matches['away_score'] = df_matches['away_score'].astype(int)

# Format 1 - Knockout round

In this simple format, we gonna put the same draw combinations. In this scenario, the debut game will be the same.
In case of a tie:
1.   The algorithm will search if this game happen on the World Cup, and have shoot out.
2.   If the both players never shootout, the algorithm will search for the last match with winner
3.   If still dont had registers, the algorithm will search for the last matches. When finded the same opponent for both, the algorithm will return the best result

## Format 1 - Execution

In [ ]:
list_teams = ["Qatar","Ecuador","Senegal","Netherlands",
         "England", "Iran", "United States", "Wales",
         "Argentina", "Saudi Arabia", "Mexico", "Poland",
         "France", "Australia", "Denmark", "Tunisia",
         "Spain", "Costa Rica", "Germany", "Japan",
         "Belgium", "Canada", "Morocco", "Croatia",
         "Brazil", "Serbia", "Switzerland", "Cameroon",
         "Portugal", "Ghana", "Uruguay", "South Korea"]

In [ ]:
Q16 = []
for i in range(0, len(list_teams), 2):
    t_winner, older_date = define_winner(df_matches.copy(),df_shootouts.copy(),list_teams[i],list_teams[i + 1],['no draw'],older_date)
    print("WINNER:",t_winner)
    Q16.append(t_winner)
print(Q16)
print("-----------------------------------------------------")
print("---------------------ROUND OF 16---------------------")
print("-----------------------------------------------------")
Q8 = []
for i in range(0, len(Q16), 2):
    t_winner, older_date = define_winner(df_matches.copy(),df_shootouts.copy(),Q16[i],Q16[i + 1],['no draw'],older_date)
    print("WINNER:",t_winner)
    Q8.append(t_winner)
print(Q8)
print("-----------------------------------------------------")
print("--------------------QUARTERFINALS--------------------")
print("-----------------------------------------------------")
SF = []
for i in range(0, len(Q8), 2):
    t_winner, older_date = define_winner(df_matches.copy(),df_shootouts.copy(),Q8[i],Q8[i + 1],['no draw'],older_date)
    print("WINNER:",t_winner)
    SF.append(t_winner)
print(SF)
print("-----------------------------------------------------")
print("----------------------SEMIFINALS---------------------")
print("-----------------------------------------------------")
finalists = []
for i in range(0, len(SF), 2):
    t_winner, older_date = define_winner(df_matches.copy(),df_shootouts.copy(),SF[i],SF[i + 1],['no draw'],older_date)
    print("WINNER:",t_winner)
    finalists.append(t_winner)
print(finalists)
print("-----------------------------------------------------")
print("------------------------FINAL------------------------")
print("-----------------------------------------------------")
t_winner, older_date = define_winner(df_matches.copy(),df_shootouts.copy(),finalists[0],finalists[1],['no draw'],older_date)
print("CHAMPION:",t_winner)




MATCH:  Qatar  X  Ecuador
Result: Qatar - 0 X 2 - Ecuador
WINNER: Ecuador
MATCH:  Senegal  X  Netherlands
Result: Senegal - 0 X 2 - Netherlands
WINNER: Netherlands
MATCH:  England  X  Iran
Result: England - 6 X 2 - Iran
WINNER: England
MATCH:  United States  X  Wales
Result: United States - 1 X 1 - Wales
Shootouts:  United States  X  Wales
United States  and  Wales  never shootouts
Historic last result:
Result: United States - 2 X 0 - Wales
Date historic: 2003-05-26
<class 'datetime.date'>
<class 'str'>
WINNER: United States
MATCH:  Argentina  X  Saudi Arabia
Result: Argentina - 1 X 2 - Saudi Arabia
WINNER: Saudi Arabia
MATCH:  Mexico  X  Poland
Result: Mexico - 0 X 0 - Poland
Shootouts:  Mexico  X  Poland
Mexico  and  Poland  never shootouts
Historic last result:
Result: Poland - 0 X 1 - Mexico
Date historic: 2017-11-13
<class 'datetime.date'>
<class 'str'>
WINNER: Mexico
MATCH:  France  X  Australia
Result: France - 4 X 1 - Australia
WINNER: France
MATCH:  Denmark  X  Tunisia
Result:

Some Fun Facts:
*   Netherlands will be Champions for the very first time
*   Belgium on the Final for the first time
*   Argentina played one only game with Saudi Arabia
*   All the teams semifinals aren't on the real semifinals World Cup 2022







# Format 2

In this format, we need to use all combination of matches, so the team will confront all the others 31 nations. In case of Victory, 3 points to the winner and 0 to de looser. Draw means each teams will recieve 1 point.



1.   The algorithm will search if this game happen on the World Cup.
2.   If the game never happen in a World Cup, the algorithm will search for other competitions, but with neutral home.
3.   In case of don't have registers of match in neutral home, the last match will be match to consider;

After all the matchs, the classification will follow this priority:
1.   Most points team
2.   Most victory teams
3.   Better SG
4.   Most goal scored





## Format 2 - Functions

In [ ]:
def return_the_other_team(row,team):
    if row['home_team'] == team:
        return row['away_team']
    else:
        return row['home_team']

In [ ]:
def in_case_no_match(df_matches,teamA, teamB):
  teams_played = []
  df_matches_a = df_matches[(df_matches["home_team"]== teamA | df_matches["away_team"]== teamA)]
  df_matches_b = df_matches[(df_matches["home_team"]== teamB | df_matches["away_team"]== teamB)]
  df_matches_a = df_matches_a.sort_values(['date'],ascending=False)
  df_matches_b = df_matches_a.sort_values(['date'],ascending=False)
  keep_going = True
  while keep_going == True:
    played_against_a = df_matches_a.apply(return_the_other_team,  args=(teamA),axis=1)
    print("Analisando:",teamA," vs ",played_against_a)
    if played_against_a not in teams_played:
      teams_played.append(played_against_a)
    else:
      team_in_comum = played_against_a
      break
    played_against_b = df_matches_b.apply(return_the_other_team,  args=(teamB),axis=1)
    print("Analisando:",teamA," vs ",played_against_a)
    if played_against_b not in teams_played:
      teams_played.append(played_against_b)
    else:
      team_in_comum = played_against_b
      break
  print("The national team who the both matched was:", team_in_comum)
  df_matches_team_a_vs_team_in_comum = df_matches[((df_matches["home_team"]== teamA) & (df_matches["away_team"]== team_in_comum)) | ((df_matches["home_team"]== team_in_comum) & (df_matches["away_team"]== teamA))]
  df_matches_team_b_vs_team_in_comum = df_matches[((df_matches["home_team"]== teamB) & (df_matches["away_team"]== team_in_comum)) | ((df_matches["home_team"]== team_in_comum) & (df_matches["away_team"]== teamB))]
  last_mactch_team_a_vs_team_in_comum = df_matches_team_a_vs_team_in_comum.sort_values(['date']).tail(1)
  last_mactch_team_b_vs_team_in_comum = df_matches_team_b_vs_team_in_comum.sort_values(['date']).tail(1)

  if  last_mactch_team_a_vs_team_in_comum['home_team']== teamA:
    teamA_score = last_mactch_team_a_vs_team_in_comum['home_score']
  else:
    teamA_score = last_mactch_team_a_vs_team_in_comum['away_score']
  if  last_mactch_team_b_vs_team_in_comum['home_team']== teamB:
    teamB_score = last_mactch_team_b_vs_team_in_comum['home_score']
  else:
    teamB_score = last_mactch_team_b_vs_team_in_comum['away_score']

  df_winner = pd.Dataframe()
  df_winner['date'] = 'NaN'
  df_winner['home_team'] = teamA
  df_winner['away_team'] = teamB
  df_winner['home_score'] = teamA_score
  df_winner['away_score'] = teamB_score

  return df_winner


## Format 2 - Execution

In [ ]:
combinations = list(itertools.combinations(list_teams, 2))
print(combinations)

[('Qatar', 'Ecuador'), ('Qatar', 'Senegal'), ('Qatar', 'Netherlands'), ('Qatar', 'England'), ('Qatar', 'Iran'), ('Qatar', 'United States'), ('Qatar', 'Wales'), ('Qatar', 'Argentina'), ('Qatar', 'Saudi Arabia'), ('Qatar', 'Mexico'), ('Qatar', 'Poland'), ('Qatar', 'France'), ('Qatar', 'Australia'), ('Qatar', 'Denmark'), ('Qatar', 'Tunisia'), ('Qatar', 'Spain'), ('Qatar', 'Costa Rica'), ('Qatar', 'Germany'), ('Qatar', 'Japan'), ('Qatar', 'Belgium'), ('Qatar', 'Canada'), ('Qatar', 'Morocco'), ('Qatar', 'Croatia'), ('Qatar', 'Brazil'), ('Qatar', 'Serbia'), ('Qatar', 'Switzerland'), ('Qatar', 'Cameroon'), ('Qatar', 'Portugal'), ('Qatar', 'Ghana'), ('Qatar', 'Uruguay'), ('Qatar', 'South Korea'), ('Ecuador', 'Senegal'), ('Ecuador', 'Netherlands'), ('Ecuador', 'England'), ('Ecuador', 'Iran'), ('Ecuador', 'United States'), ('Ecuador', 'Wales'), ('Ecuador', 'Argentina'), ('Ecuador', 'Saudi Arabia'), ('Ecuador', 'Mexico'), ('Ecuador', 'Poland'), ('Ecuador', 'France'), ('Ecuador', 'Australia'), ('E

In [ ]:
df_table = pd.DataFrame({'team':list_teams})
df_table = df_table.set_index(pd.Index(range(1, len(df_table) + 1)))
df_table["points"] = 0
df_table["matches"] = 0
df_table["wins"] = 0
df_table["draws"] = 0
df_table["loose"] = 0
df_table["sg"] = 0
df_table["gp"] = 0
df_table["gc"] = 0
df_table["status"] = "On competition"
df_table


,team,points,matches,wins,draws,loose,sg,gp,gc,status
1,Qatar,0,0,0,0,0,0,0,0,On competition
2,Ecuador,0,0,0,0,0,0,0,0,On competition
3,Senegal,0,0,0,0,0,0,0,0,On competition
4,Netherlands,0,0,0,0,0,0,0,0,On competition
5,England,0,0,0,0,0,0,0,0,On competition
6,Iran,0,0,0,0,0,0,0,0,On competition
7,United States,0,0,0,0,0,0,0,0,On competition
8,Wales,0,0,0,0,0,0,0,0,On competition
9,Argentina,0,0,0,0,0,0,0,0,On competition
10,Saudi Arabia,0,0,0,0,0,0,0,0,On competition


In [ ]:
combinations[3][1]

'England'

In [ ]:
for teamA, teamB in combinations:
    #print(f"{teamA} e {teamB}")
    t_winner, older_date = define_winner(df_matches.copy(),df_shootouts.copy(),teamA, teamB,["same opponent"],older_date)


In [ ]:
for teams in combinations:
    for team in teams:
        print(team)